# GenAI Complex Reasoning / Chain of Thought

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.bigquery', 'google-cloud-bigquery')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [5]:
REGION = 'us-central1'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

packages:

In [6]:
import os
import json
import io
import base64
import asyncio
import requests
import IPython
import datetime, time

from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

print('complete')

complete


In [7]:
aiplatform.__version__

'1.67.1'

clients:

In [8]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

---
## Vertex AI Package

With the [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) client there are packages for the types of data being interacted with.  There is also a higher package for preview models (not yet in GA).

> **NOTE:** In can be helpful to review the API Documentation at it's source in GitHub for up to the moment release information: [github/googleapis/python-aiplatform](https://github.com/googleapis/python-aiplatform/tree/main)

Gemini Text and Multimodal Models:
- [vertexai.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models)
    - [vertexai.preview.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.generative_models)

Language Models (PaLM and Codey Models):
- [vertexai.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models)
    - [vertexai.preview.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models)

Vision Models (Imagen Models):
- [vertexai.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.vision_models)
    - [vertexai.preview.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.vision_models)


In [9]:
# Gemini Models
#gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-002")
#gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision-001")
#gemini15_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")
gemini_flash = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")


## Reasoning flow

In [17]:
def make_api_call(prompt, max_tokens, is_final_answer=False, custom_client=None):
    global client
    if custom_client != None:
        client = gemini_flash
    
    for attempt in range(3):
        try:
            if is_final_answer:
                response = client.generate_content(prompt)
                return response.text
            else:
                response = client.generate_content(prompt)
                return response.text
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error", "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}", "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying


In [ ]:
def generate_response(user_prompt, custom_client=None):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant that explains your reasoning step by step. For each step, provide a title that describes what you're doing in that step, along with the content. Decide if you need another step or if you're ready to give the final answer. Respond in JSON format with 'title', 'content', and 'next_action' (either 'continue' or 'final_answer') keys. USE AS MANY REASONING STEPS AS POSSIBLE. AT LEAST 3. BE AWARE OF YOUR LIMITATIONS AS AN LLM AND WHAT YOU CAN AND CANNOT DO. IN YOUR REASONING, INCLUDE EXPLORATION OF ALTERNATIVE ANSWERS. CONSIDER YOU MAY BE WRONG, AND IF YOU ARE WRONG IN YOUR REASONING, WHERE IT WOULD BE. FULLY TEST ALL OTHER POSSIBILITIES. YOU CAN BE WRONG. WHEN YOU SAY YOU ARE RE-EXAMINING, ACTUALLY RE-EXAMINE, AND USE ANOTHER APPROACH TO DO SO. DO NOT JUST SAY YOU ARE RE-EXAMINING. USE AT LEAST 3 METHODS TO DERIVE THE ANSWER. USE BEST PRACTICES.

Example of a valid JSON response:
```json
{
    "title": "Identifying Key Information",
    "content": "To begin solving this problem, we need to carefully examine the given information and identify the crucial elements that will guide our solution process. This involves...",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]
    
    steps = []
    step_count = 1
    total_thinking_time = 0
    
    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300, custom_client=custom_client)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time
        
        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))
        
        messages.append({"role": "assistant", "content": json.dumps(step_data)})
        
        if step_data['next_action'] == 'final_answer' or step_count > 25: # Maximum of 25 steps to prevent infinite thinking time. Can be adjusted.
            break
        
        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based solely on your reasoning above. Do not use JSON formatting. Only provide the text response without any titles or preambles. Retain any formatting as instructed by the original prompt, such as exact formatting for free response or multiple choice."})
    
    start_time = time.time()
    final_data = make_api_call(messages, 1200, is_final_answer=True, custom_client=custom_client)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time
    
    steps.append(("Final Answer", final_data, thinking_time))

    yield steps, total_thinking_time

### Text Prompt

In [10]:
prompts = [
    "What is a mulligan?",
    "What do you do when you have an unplayable lie in golf?"
]

In [13]:
response = gemini_flash.generate_content(prompts[1])
#response

In [14]:
print(response.text)

When you have an unplayable lie in golf, you have several options, all resulting in a penalty stroke:

1. **Play the ball as it lies:** This is always an option, though rarely the best one if the lie is truly unplayable.  You'll just have to hit it from where it sits.

2. **Take relief under Rule 19:** This is the most common approach for unplayable lies. You can take relief in one of the following ways:

    * **One club-length relief:**  Drop the ball within one club-length of where the ball lies, not nearer the hole.
    * **Two club-lengths relief:** Drop the ball within two club-lengths of where the ball lies, not nearer the hole.  This is only available if dropping within one club length is impossible due to interference from a hazard or other obstruction.
    * **Drop from a point nearer the hole:**  You can drop the ball from a spot nearer the hole, losing one stroke.  This is useful if you have no playable lie within a reasonable distance of the original spot, even considering

In [15]:
IPython.display.Markdown(response.text)

When you have an unplayable lie in golf, you have several options, all resulting in a penalty stroke:

1. **Play the ball as it lies:** This is always an option, though rarely the best one if the lie is truly unplayable.  You'll just have to hit it from where it sits.

2. **Take relief under Rule 19:** This is the most common approach for unplayable lies. You can take relief in one of the following ways:

    * **One club-length relief:**  Drop the ball within one club-length of where the ball lies, not nearer the hole.
    * **Two club-lengths relief:** Drop the ball within two club-lengths of where the ball lies, not nearer the hole.  This is only available if dropping within one club length is impossible due to interference from a hazard or other obstruction.
    * **Drop from a point nearer the hole:**  You can drop the ball from a spot nearer the hole, losing one stroke.  This is useful if you have no playable lie within a reasonable distance of the original spot, even considering the one or two club-length options.

3. **Take relief under other rules:** In rare circumstances, your unplayable lie might be affected by other rules, such as an obstruction that isn't part of the course. You would need to apply the appropriate rule in that case.

**Important Considerations:**

* **Defining "Unplayable":**  This is subjective.  If you believe it's impossible to play a reasonable shot from your lie, then it's considered unplayable.

* **Penalty:**  Regardless of the method you choose (except playing it as it lies), you incur a one-stroke penalty.

* **Free Drop:**  When taking relief, you must ensure that the ball is dropped from waist height within the specified area and doesn't come to rest in a worse location.  You can drop it multiple times if needed to comply.

* **Local Rules:** Always check the scorecard or local rules for any specific regulations regarding unplayable lies on that particular course.

In short, if your lie is truly unplayable, take a penalty stroke and choose the relief option that gives you the best chance to play your next shot effectively.  Remember to consult the Rules of Golf for a thorough understanding.


#### Streaming Response

In [15]:
for response in gemini_text.generate_content(prompts[1], stream = True):
    print(response.text)

There
 are a few options available when you find yourself with an unplayable lie in golf:


**Take an unplayable lie penalty:** This is the most common option. You will
 add one stroke to your score and drop the ball in a designated relief area. The relief area is typically two club-lengths from the spot where the unplayable lie occurred
, no closer to the hole. You must drop the ball within the relief area, not closer to the hole than the original position.

**Hit a provisional
 ball:** Before deciding the lie is unplayable, you can hit a provisional ball from a different location (often the fairway or rough) where you have a playable lie. If you find the original ball and confirm the lie is unplayable, you can
 take a one-stroke penalty and play the provisional ball. If you find the original ball and it's playable, you can continue playing it and discard the provisional ball.

**Declare the ball lost:** If you are unable to find the ball or
 believe it is lost, you can declare it los

#### Async Response

The client has built in method for awaitable responses that make it easy to make asynchronous request.

> For detailed coverage and examples of asynchronous call to these API's, scaling, error handling, and managing fail over regions check out this notebook in the same folder: [Python Asynchronous API Calls](./Python%20Asynchronous%20API%20Calls.ipynb)

In [16]:
response = await gemini_text.generate_content_async(prompts[1])
#print(response.text)
IPython.display.Markdown(response.text)

## Unplayable Lies in Golf

An unplayable lie in golf can be frustrating, but don't worry, there are options! Here's what you can do:

**1. Take a Penalty Stroke:**

This is the simplest option. You can take a one-stroke penalty and drop the ball within two club lengths of the original spot, no closer to the hole. Keep in mind that these two club lengths should be measured in any direction, except towards the hole.

**2. Play the Ball as it Lies:**

While challenging, you can attempt to play the ball as it lies. This could be a good option if you're confident in your ability to hit a difficult shot or if the penalty stroke would put you in an even worse position.

**3. Take Lateral Relief:**

If your ball is in a penalty area (water hazard or out-of-bounds), you can take lateral relief. This means dropping the ball within two club lengths of the point where the ball crossed the margin of the penalty area, but not closer to the hole. There is a two-stroke penalty for taking lateral relief.

**4. Provisional Ball:**

Before hitting your original shot, you can hit a provisional ball in case your original shot ends up being unplayable. You can then play the provisional ball without penalty if your original shot is deemed unplayable.

**Additional Considerations:**

* Always check the local rules before determining how to proceed with an unplayable lie.
* Consider the conditions and your own skill level when choosing the best option for your situation.
* Remember to inform your playing partners and marker when taking relief or playing a provisional ball.

Here are some helpful resources:

* **USGA Rule 19:** https://www.usga.org/content/dam/usga/pdf/2023/rule-books/rules-of-golf.pdf
* **R&A Rule 19:** https://www.randa.org/en/rules-and-amateur-status/rules-of-golf
* **PGA Instruction on Unplayable Lie:** https://www.pga.com/practice/golf-tips/how-to-play-an-unplayable-lie

I hope this helps! Let me know if you have any other questions about golf or anything else.

In [ ]:
prompt_system = """
Begin by enclosing all thoughts within <thinking> tags, exploring multiple angles and approaches.
Break down the solution into clear steps within <step> tags. Start with a 20-step budget, requesting more for complex problems if needed.
Use <count> tags after each step to show the remaining budget. Stop when reaching 0.
Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress.
Regularly evaluate progress using <reflection> tags. Be critical and honest about your reasoning process.
Assign a quality score between 0.0 and 1.0 using <reward> tags after each reflection. Use this to guide your approach:

0.8+: Continue current approach
0.5-0.7: Consider minor adjustments
Below 0.5: Seriously consider backtracking and trying a different approach


If unsure or if reward score is low, backtrack and try a different approach, explaining your decision within <thinking> tags.
For mathematical problems, show all work explicitly using LaTeX for formal notation and provide detailed proofs.
Explore multiple solutions individually if possible, comparing approaches in reflections.
Use thoughts as a scratchpad, writing out all calculations and reasoning explicitly.
Synthesize the final answer within <answer> tags, providing a clear, concise summary.
Conclude with a final reflection on the overall solution, discussing effectiveness, challenges, and solutions. Assign a final reward score.
"""

In [ ]:
def make_api_call(messages, max_tokens, is_final_answer=False, custom_client=None):
    global client
    if custom_client != None:
        client = custom_client
    
    for attempt in range(3):
        try:
            if is_final_answer:
                response = client.chat.completions.create(
                    model="llama-3.1-70b-versatile",
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=0.2,
            ) 
                return response.choices[0].message.content
            else:
                response = client.chat.completions.create(
                    model="llama-3.1-70b-versatile",
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=0.2,
                    response_format={"type": "json_object"}
                )
                return json.loads(response.choices[0].message.content)
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error", "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}", "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response(prompt, custom_client=None):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant that explains your reasoning step by step. For each step, provide a title that describes what you're doing in that step, along with the content. Decide if you need another step or if you're ready to give the final answer. Respond in JSON format with 'title', 'content', and 'next_action' (either 'continue' or 'final_answer') keys. USE AS MANY REASONING STEPS AS POSSIBLE. AT LEAST 3. BE AWARE OF YOUR LIMITATIONS AS AN LLM AND WHAT YOU CAN AND CANNOT DO. IN YOUR REASONING, INCLUDE EXPLORATION OF ALTERNATIVE ANSWERS. CONSIDER YOU MAY BE WRONG, AND IF YOU ARE WRONG IN YOUR REASONING, WHERE IT WOULD BE. FULLY TEST ALL OTHER POSSIBILITIES. YOU CAN BE WRONG. WHEN YOU SAY YOU ARE RE-EXAMINING, ACTUALLY RE-EXAMINE, AND USE ANOTHER APPROACH TO DO SO. DO NOT JUST SAY YOU ARE RE-EXAMINING. USE AT LEAST 3 METHODS TO DERIVE THE ANSWER. USE BEST PRACTICES.

Example of a valid JSON response:
```json
{
    "title": "Identifying Key Information",
    "content": "To begin solving this problem, we need to carefully examine the given information and identify the crucial elements that will guide our solution process. This involves...",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]
    
    steps = []
    step_count = 1
    total_thinking_time = 0
    
    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300, custom_client=custom_client)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time
        
        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))
        
        messages.append({"role": "assistant", "content": json.dumps(step_data)})
        
        if step_data['next_action'] == 'final_answer' or step_count > 25: # Maximum of 25 steps to prevent infinite thinking time. Can be adjusted.
            break
        
        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based solely on your reasoning above. Do not use JSON formatting. Only provide the text response without any titles or preambles. Retain any formatting as instructed by the original prompt, such as exact formatting for free response or multiple choice."})
    
    start_time = time.time()
    final_data = make_api_call(messages, 1200, is_final_answer=True, custom_client=custom_client)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time
    
    steps.append(("Final Answer", final_data, thinking_time))

    yield steps, total_thinking_time